In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!huggingface-cli login



    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) y
Token is valid (permission: fineGrained).
The token `1` has been saved to /root/.cache/huggingface/stored_tokens
Cannot authenticate through git-credential as no helper is defined on your machine.
You might have to re-authenticate when p

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os

In [ ]:
!pip install datasets


In [ ]:
from datasets import load_dataset

# Load the vqa-rad dataset
dataset = load_dataset("mdwiratathya/SLAKE-vqa-english")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/1.11k [00:00<?, ?B/s]

train-00000-of-00002.parquet:   0%|          | 0.00/31.1M [00:00<?, ?B/s]

train-00001-of-00002.parquet:   0%|          | 0.00/12.2M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/8.34M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/9.59M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/4919 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1053 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1061 [00:00<?, ? examples/s]

In [ ]:
import torch
import io
import pickle
import pandas as pd
from transformers import BlipProcessor, BlipForQuestionAnswering
from torch.utils.data import DataLoader
from tqdm import tqdm
from PIL import Image
from transformers import AutoModelForCausalLM, BitsAndBytesConfig


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
!pip install peft

In [ ]:
!pip install -U bitsandbytes


In [ ]:
!pip install -qU transformers

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from transformers import AutoProcessor, AutoModelForVision2Seq,AutoModel
from peft import LoraConfig  # Import LoRA configuration
from transformers import AutoModelForCausalLM

# Check if LoRA or QLoRA should be used

    # Define LoRA configuration
lora_config = LoraConfig(
r=8,
lora_alpha=8,
lora_dropout=0.1,
target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],  # Adjust based on inspected names
bias="none",
init_lora_weights="gaussian"
)

    # If using QLoRA, configure Bits and Bytes
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16
)


    # Load model with quantization configuration if using QLoRA
from transformers import AutoProcessor, AutoModelForPreTraining

processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
model = AutoModelForPreTraining.from_pretrained("llava-hf/llava-1.5-7b-hf",
    torch_dtype=torch.float16,
    quantization_config=bnb_config,
)

# Add LoRA configuration to the model
model.add_adapter(lora_config)
model.enable_adapters()


# Move model to the correct device


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


processor_config.json:   0%|          | 0.00/173 [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/505 [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.48, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

model-00001-of-00003.safetensors:   0%|          | 0.00/4.99G [00:00<?, ?B/s]

model-00002-of-00003.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00003-of-00003.safetensors:   0%|          | 0.00/4.18G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/141 [00:00<?, ?B/s]

In [ ]:
!pip install datasets

In [ ]:
from datasets import load_dataset

dataset = load_dataset("mdwiratathya/SLAKE-vqa-english")
dataset['train'] = dataset['train'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)
dataset['test'] = dataset['test'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)
dataset['validation'] = dataset['validation'].filter(lambda example: example['question'] is not None and example['answer'] is not None and example['image'] is not None)


Filter:   0%|          | 0/4919 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1061 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1053 [00:00<?, ? examples/s]

In [ ]:
# Inspect the dataset for None values
for idx, example in enumerate(dataset['test']):
    if example['question'] is None or example['image'] is None or example['answer'] is None:
        print(f"Missing values in example at index {idx}: {example}")


In [ ]:
from datasets import load_dataset
from torch.utils.data import Dataset
from PIL import Image
import random
processor = AutoProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")

# Define the VQADataset class
class VQADataset(Dataset):
    def __init__(self, dataset, processor):
        self.dataset = dataset
        self.processor = processor

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        question = self.dataset[idx]['question']
        answer = self.dataset[idx]['answer']
        image = self.dataset[idx]['image']  # Assuming it's a PIL image

        return {
            "image": image,
            "query": {"en": question},
            "answers": [answer]
        }


# Define the MyDataCollator class
class MyDataCollator:
    def __init__(self, processor):
        self.processor = processor
        self.image_token = '<image>'
        self.end_of_utterance_token = '<end_of_utterance>'
        self.pad_token_id = getattr(processor, 'pad_token_id', 0)  # Default to 0 if not found

    def __call__(self, examples):
        texts = []
        for example in examples:
            question = example["query"]['en']
            answer = random.choice(example["answers"])

            messages = [
                {
                    "role": "user",
                    "content": [
                        {"type": "text", "text": "Answer briefly."},
                        {"type": "text", "text": self.image_token},
                        {"type": "text", "text": question}
                    ]
                },
                {
                    "role": "assistant",
                    "content": [
                        {"type": "text", "text": answer},
                        {"type": "text", "text": self.end_of_utterance_token}
                    ]
                }
            ]

            text = self.processor.apply_chat_template(messages, add_generation_prompt=False)

            if isinstance(text, list):
                text = " ".join(str(item) for item in text)
            else:
                text = str(text)

            texts.append(text.strip())

        text_batch = self.processor(text=texts, return_tensors="pt", padding=True)

        # Prepare the batch dictionary
        batch = {
            "input_ids": text_batch["input_ids"],
            "attention_mask": text_batch["attention_mask"]
        }

        # Create labels
        labels = batch["input_ids"].clone()
        labels[labels == self.pad_token_id] = -100  # Ignore padding in loss calculation
        batch["labels"] = labels

        return batch




# Create dataset objects for each split
train_dataset = VQADataset(dataset=dataset['train'], processor=processor)
test_dataset = VQADataset(dataset=dataset['test'], processor=processor)
valid_dataset = VQADataset(dataset=dataset['validation'], processor=processor)

# Data collator for batching
data_collator = MyDataCollator(processor=processor)


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir = "LLavaMed",
    learning_rate = 2e-4,
    fp16 = True,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 8,
    dataloader_pin_memory = False,
    save_total_limit = 3,
    evaluation_strategy ="steps",
    save_strategy = "steps",
    eval_steps = 100,
    save_steps = 100,
    max_steps = 500,
    logging_steps = 5,
    remove_unused_columns = False,
    push_to_hub=False,
    label_names = ["labels"],
    load_best_model_at_end = False,
    report_to = "none",
    optim = "paged_adamw_8bit",
    weight_decay=0.05
)

/usr/local/lib/python3.11/dist-packages/transformers/training_args.py:1594: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    data_collator=data_collator,
    train_dataset = train_dataset,
    eval_dataset = valid_dataset
)

In [ ]:
trainer.train()


Step,Training Loss,Validation Loss
100,4.476800,0.514324
200,3.548500,0.415252
300,3.230500,0.375819
400,2.683800,0.351915
500,2.482000,0.342534


TrainOutput(global_step=500, training_loss=4.975839281082154, metrics={'train_runtime': 3019.1068, 'train_samples_per_second': 1.325, 'train_steps_per_second': 0.166, 'total_flos': 5936572519114752.0, 'train_loss': 4.975839281082154, 'epoch': 0.8131734092295182})

In [ ]:
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer,LlavaProcessor

#32
save_directory = "/content/drive/MyDrive/VQA12Feb/models/llavaFinalslakee"

from transformers import AutoConfig

config = AutoConfig.from_pretrained("llava-hf/llava-1.5-7b-hf")  # Use the base model
config.save_pretrained(save_directory)
model.save_pretrained(save_directory)
# processor = LlavaProcessor.from_pretrained("llava-hf/llava-1.5-7b-hf")
processor.save_pretrained(save_directory)

print(f"✅ Model saved at: {save_directory}")

✅ Model saved at: /content/drive/MyDrive/VQA12Feb/models/llavaFinalslakee


In [ ]:
from huggingface_hub import HfApi

# Set up the model and tokenizer save path
save_directory = "/content/drive/MyDrive/VQA12Feb/models/llavaFinalslakee"
repo_name = "llava20FeSlakee"
user_name = "Dtarget"

api = HfApi()

# Create the repository as private
api.create_repo(repo_id=f"{user_name}/{repo_name}", private=True, repo_type="model")

api.upload_folder(
    folder_path=save_directory,
    repo_id=f"{user_name}/{repo_name}",
    path_in_repo=""  # Upload to root of the repository
)

print(f"✅ Model uploaded to Hugging Face privately: https://huggingface.co/{user_name}/{repo_name}")


adapter_model.safetensors:   0%|          | 0.00/38.3M [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

✅ Model uploaded to Hugging Face privately: https://huggingface.co/Dtarget/llava20FeSlakee


In [ ]:


# Evaluate the model
results = trainer.evaluate()

print(results)


{'eval_loss': 0.3425341546535492, 'eval_runtime': 240.5924, 'eval_samples_per_second': 4.377, 'eval_steps_per_second': 4.377, 'epoch': 0.8131734092295182}


In [ ]:
test_results = trainer.evaluate(test_dataset)

print(test_results)

{'eval_loss': 0.3383178412914276, 'eval_runtime': 242.6478, 'eval_samples_per_second': 4.373, 'eval_steps_per_second': 4.373, 'epoch': 0.8131734092295182}
